In [8]:
import os
import struct
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm
from pathlib import Path
from hloc import extract_features, match_features, reconstruction,pairs_from_retrieval


import pycolmap
import time
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster
import json


import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superpoint+lightglue']

# Force GPU usage
retrieval_conf['device'] = 'cuda'  # Ensure NetVLAD uses GPU
feature_conf['device'] = 'cuda'    # Ensure SuperPoint uses GPU
matcher_conf['device'] = 'cuda'    # Ensure LightGlue uses GPU

In [10]:
# Paths
images = Path('/media/siyanhu/T71/hloc/datasets/')
outputs = Path('/media/siyanhu/T71/hloc/dji_recon')

sfm_pairs = outputs / 'pairs-netvlad.txt'
loc_pairs = outputs / 'pairs-loc.txt'

sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
retrieval_features = outputs/'features_retrieval.h5'

In [11]:
ref_seqs = [7, 8, 9]

ref_paths = []

for i in range(9):
    seq_num = i + 1
    if seq_num in ref_seqs:
        for p in (images / f'rgb{seq_num}/').iterdir():
            if '.jpg' in str(p.relative_to(images)):
                    ref_paths.append(str(p.relative_to(images)))
    
print(ref_paths[5:10], ref_paths[-10:-5])

['rgb7/00006.jpg', 'rgb7/00007.jpg', 'rgb7/00008.jpg', 'rgb7/00009.jpg', 'rgb7/00010.jpg'] ['rgb9/00747.jpg', 'rgb9/00748.jpg', 'rgb9/00749.jpg', 'rgb9/00750.jpg', 'rgb9/00751.jpg']


In [12]:
retrieval_path = extract_features.main(retrieval_conf, images, image_list = ref_paths, feature_path = retrieval_features)
print(retrieval_path)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=20)

[2025/03/16 19:51:11 hloc INFO] Extracting local features with configuration:
{'device': 'cuda',
 'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}


  0%|          | 0/2268 [00:00<?, ?it/s]

[2025/03/16 19:55:00 hloc INFO] Finished exporting features.
[2025/03/16 19:55:00 hloc INFO] Extracting image pairs from a retrieval database.


/media/siyanhu/T71/hloc/dji_recon/features_retrieval.h5


[2025/03/16 19:55:01 hloc INFO] Found 45360 pairs.


In [ ]:
extract_features.main(feature_conf, images, image_list=ref_paths, feature_path=features)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

[2025/03/16 19:56:36 hloc INFO] Extracting local features with configuration:
{'device': 'cuda',
 'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}


Loaded SuperPoint model


/home/siyanhu/Gits/Hierarchical-Localization/hloc/extractors/../../third_party/SuperGluePretrainedNetwork/models/superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

  0%|          | 0/2268 [00:00<?, ?it/s]

[2025/03/16 19:57:45 hloc INFO] Finished exporting features.
[2025/03/16 19:57:45 hloc INFO] Matching local features with configuration:
{'device': 'cuda',
 'model': {'features': 'superpoint', 'name': 'lightglue'},
 'output': 'matches-superpoint-lightglue'}
/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/site-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


  0%|          | 0/25151 [00:00<?, ?it/s]

Exception in thread Thread-12 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/home/siyanhu/miniconda3/envs/py311hloc_backup/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/siyanhu/miniconda3/envs/py31

KeyboardInterrupt: 

: 

In [25]:
print(sfm_dir / "database.db")

/media/siyanhu/T71/hloc/ref_recon/sfm/database.db


In [ ]:
model = reconstruction.main(
    sfm_dir,
    images,
    sfm_pairs,
    features,
    matches,
    image_list=ref_paths
)

[2025/03/14 14:43:25 hloc INFO] Performing geometric verification of the matches...
I20250314 14:43:25.396557 126454524282560 misc.cc:44] 
Feature matching
I20250314 14:43:25.404977 126453890942656 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.404993 126453725267648 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405001 126453983217344 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405078 126453880456896 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405079 126453869971136 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405089 126454025160384 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405096 126454318761664 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405115 126453993703104 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405136 126453859485376 sift.cc:1432] Creating SIFT CPU feature matcher
I20250314 14:43:25.405087 12645390

In [50]:
references_registered_init = [model.images[i].name for i in model.reg_image_ids()]
print(references_registered_init[5:10], references_registered_init[-10:-5])

['rgb1/00006.jpg', 'rgb1/00007.jpg', 'rgb1/00008.jpg', 'rgb1/00009.jpg', 'rgb1/00010.jpg'] ['rgb9/00747.jpg', 'rgb9/00748.jpg', 'rgb9/00749.jpg', 'rgb9/00750.jpg', 'rgb9/00751.jpg']


In [56]:
references_registered = []
skip_list = ['rgb7', 'rgb8', 'rgb9']

for name in references_registered_init:
    prefix = name.split('/')[0]
    if prefix not in skip_list:
        references_registered.append(name)

print(references_registered[5:10], references_registered[-10:-5])

['rgb1/00006.jpg', 'rgb1/00007.jpg', 'rgb1/00008.jpg', 'rgb1/00009.jpg', 'rgb1/00010.jpg'] ['rgb3/00746.jpg', 'rgb3/00747.jpg', 'rgb3/00748.jpg', 'rgb3/00749.jpg', 'rgb3/00750.jpg']


In [3]:
query_list = []
for i in range(7, 10):
    for p in (images / f'rgb{i}/').iterdir():
        if '.jpg' in str(p.relative_to(images)):
            query_list.append(str(p.relative_to(images)))

print(query_list[5:10], query_list[-10:-5])

NameError: name 'images' is not defined

In [2]:
ret_list = []
log_list = []
pose_dict = {}
time_total = 0

def parse_retrieval(path):
    retrieval = []
    with open(path, "r") as f:
        for p in f.read().rstrip("\n").split("\n"):
            if len(p) == 0:
                continue
            q, r = p.split()
            retrieval.append(r)
    return retrieval

In [4]:
count = 0
for query in query_list:
    count += 1
    print('progress: ',count/len(query_list), 'query: ', query)
    # try:
    time_start = time.time()
    extract_features.main(feature_conf, images, image_list=[query], feature_path=features, overwrite=True)
    global_descriptors = extract_features.main(retrieval_conf, images, image_list=[query], feature_path=retrieval_features)
    pairs_from_retrieval.main(global_descriptors, loc_pairs, num_matched=20, db_list = references_registered, query_list=[query])
    match_features.main(matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True)
    retrieval_images = parse_retrieval(loc_pairs)
    camera = pycolmap.infer_camera_from_image(images/ query)
    ref_ids = []
    for n in references_registered:
        if n in retrieval_images:
            ref_ids.append(model.find_image_with_name(n).image_id)
    print(ref_ids)
    
    conf = {
        'estimation': {'ransac': {'max_error': 12}},
        'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
    }
    localizer = QueryLocalizer(model, conf)
    ret, log = pose_from_cluster(localizer, query, camera, ref_ids, features, matches)
    #print(ret)
    time_end = time.time()
    inference_time = time_end - time_start
    pose_dict[query] = [ret['cam_from_world'].rotation.quat.tolist(),ret['cam_from_world'].translation.tolist(),inference_time]
    time_total += inference_time
    # except Exception as e:
    #     print("============================================")
    #     print(e)
    #     print("============================================")
    #     pass
    #print('inference_time',inference_time)
    
print('avg inference time: ', time_total/len(query_list))
json_str = json.dumps(pose_dict)
with open(str(outputs)+'_query_results.json', 'w') as json_file:
    json_file.write(json_str)

ZeroDivisionError: division by zero